# Load packages

In [ ]:
import tensorflow as tf

tf.compat.v1.disable_v2_behavior()
tf.get_logger().setLevel('ERROR')

import json
import math
import numpy as np
import matplotlib.pyplot as plt
import gc

In [ ]:
!pip install tensorflow_privacy

from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer

# Load MNIST digits and rotate

In [ ]:
train, test = tf.keras.datasets.mnist.load_data()
train_data, train_labels = train
test_data, test_labels = test

train_data = np.array(train_data, dtype=np.float32) / 255
test_data = np.array(test_data, dtype=np.float32) / 255

train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

train_labels = np.array(train_labels, dtype=np.int32)
test_labels = np.array(test_labels, dtype=np.int32)

In [ ]:
# Grab the 6000 train examples for each digit
train_data_1, train_data_1_rot = np.split(train_data[np.where(train_labels == 1)][0:6000], 2)
train_data_7, train_data_7_rot = np.split(train_data[np.where(train_labels == 7)][0:6000], 2)

# Rotate half of them 90 degrees
train_data_1_rot = np.rot90(train_data_1_rot, axes=(1,2))
train_data_7_rot = np.rot90(train_data_7_rot, axes=(1,2))

# Original and non-rotated test digits
test_data_1 = test_data[np.where(test_labels == 1)][0:1000]
test_data_7 = test_data[np.where(test_labels == 7)][0:1000]

test_data_1_rot = np.rot90(test_data_1, axes=(1,2))
test_data_7_rot = np.rot90(test_data_7, axes=(1,2))

In [ ]:
cat_labels_1 = np.repeat(np.array([[1, 0]], dtype=np.int32), 6000, axis=0)
cat_labels_7 = np.repeat(np.array([[0, 1]], dtype=np.int32), 6000, axis=0)

# Generate grid of models to train

In [ ]:
!mkdir -p rotated_v2__1_72
rootdir = 'rotated_v2__1_72/'
batch_size = 200
num_microbatches = 200

In [ ]:
model_grid = []

dataset_sizes = [math.ceil(6000/d/batch_size)*batch_size for d in [1, 1.5, 2, 3, 4, 6, 8, 12, 16, 32]]
aVals = [1/32, 1/16, 1/8, .25, .5, 1, 2, 4, 8, 16, 32]
minority_percents = [0.0, .05, .10, .15, .20, .25, .30, .35, .40, .45, .50]
run_numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 16, 20, 21, 22]

for run_number in run_numbers:
  for dataset_size in dataset_sizes:
    for aVal in aVals:
      for minority_percent in minority_percents:
        model_grid.append({
          'run_number': run_number,
          'dataset_size': dataset_size,
          'aVal': aVal,
          'minority_percent': minority_percent,
          'l2_norm_clip': 1.5*aVal,
          'noise_multiplier': 1.3/aVal,
          'epochs': 4,
        })

for m in model_grid:
  m['slug'] = 'grid__runnumber_' + str(m['run_number']) + 'datasetsize_' + str(m['dataset_size']) + '__minority_percent_' + str(m['minority_percent']) + '__l2_norm_clip_' + str(m['l2_norm_clip']) + '__noise_multiplier_' + str(m['noise_multiplier']) + '__epochs_' + str(m['epochs'])

In [ ]:
dataset_sizes

# Train models

In [ ]:
def calc_model(m):
  path = rootdir + m['slug']

  # skip models with existing test_predictions
  try:
    test_path = path + '___test_data_7_rot.npy'
    print(test_path)
    with open(test_path, 'r') as fh:
      return
  except Exception as e:
    print('no cache, training')
    print(m)

  if batch_size % num_microbatches != 0:
    raise ValueError('Batch size should be an integer multiple of the number of microbatches')

  model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu',
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2)
  ])

  optimizer = DPKerasSGDOptimizer(
    l2_norm_clip=m['l2_norm_clip'],
    noise_multiplier=m['noise_multiplier'],
    num_microbatches=num_microbatches,
    learning_rate=0.25)

  loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)
  
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  p = m['minority_percent']
  n = m['dataset_size']
  n_straight = int(.5*n*(1 - p))
  n_rot = int(.5*n*p)

  m_train_data = np.concatenate((
    train_data_1[0:n_straight],
    train_data_1_rot[0:n_rot],
    train_data_7[0:n_straight],
    train_data_7_rot[0:n_rot],
  ))

  m_train_labels = np.concatenate((
    cat_labels_1[0:int(.5*n)],
    cat_labels_7[0:int(.5*n)],
  ))
  model.fit(
    m_train_data, 
    m_train_labels,
    epochs=m['epochs'],
    batch_size=batch_size)
  
  model.save(path)

  def run_inference(data, slug, target_index):
    predictions = model.predict(data)
    percents = tf.compat.v2.nn.softmax(predictions)
    percents = percents.eval(session=tf.compat.v1.Session())
    with open(path + '___' + slug + '.npy', 'w') as fh:
      np.save(fh, percents)

    count = 0 
    for d in percents:
      if (d[target_index] > .5):
        count = count + 1
    print(slug, count)


  run_inference(test_data_1, 'test_data_1', 0)
  run_inference(test_data_7, 'test_data_7', 1)

  run_inference(test_data_1_rot, 'test_data_1_rot', 0)
  run_inference(test_data_7_rot, 'test_data_7_rot', 1)

  # Fixes OOM errors
  del m_train_data  
  del m_train_labels
  del model
  
  gc.collect()
  tf.keras.backend.clear_session()

In [ ]:
for m in model_grid:
  calc_model(m)

# Calculate metadata

In [ ]:
# Calculate model accuracy
for m in model_grid:
  path = rootdir + m['slug']

  def calc_percents(data, slug, target_index):
    try:
      with gfile.GFile(path + '___' + slug + '.npy', 'r') as fh:
        percents = np.load(fh)
    except Exception as e:
      # print(e)
      return print('missing ' + path + '___' + slug + '.npy')

    count = 0 
    for d in percents:
      if (d[target_index] > .5):
        count = count + 1
    m['accuracy_' + slug] = count


  calc_percents(test_data_1, 'test_data_1', 0)
  calc_percents(test_data_7, 'test_data_7', 1)

  calc_percents(test_data_1_rot, 'test_data_1_rot', 0)
  calc_percents(test_data_7_rot, 'test_data_7_rot', 1)



In [ ]:
from functools import lru_cache

@lru_cache(maxsize=None)
def computeModelPrivacy(dataset_size, noise_multiplier, epochs):
  eps, delta = compute_dp_sgd_privacy.compute_dp_sgd_privacy(
    n=dataset_size,
    batch_size=batch_size,
    noise_multiplier=noise_multiplier,
    epochs=epochs,
    delta=1e-5)
  
  return eps

In [ ]:
# Calculate epsilon for each model
for m in model_grid:
  m['epsilon'] = computeModelPrivacy(m['dataset_size'], m['noise_multiplier'], m['epochs'])

In [ ]:
import pandas as pd

df = pd.DataFrame(model_grid).drop(['slug'], axis=1).to_csv(rootdir + 'model_grid_train_accuracy.csv', index=False)

In [ ]:
%download_file rotated_v2__1_72/model_grid_train_accuracy.csv